In [ ]:
# Importing necessary packages:
#import re
#import os
from glob import glob
#import math
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
from matplotlib import rcParams, cycler
import matplotlib.lines as lines
from collections import OrderedDict
from matplotlib.legend import Legend
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.collections import PolyCollection # for chaing confidence interval color in statsmodels.graphics.tsaplots.plot_acf

import seaborn as sns

import statsmodels.tsa.stattools as tsas
import statsmodels.graphics.tsaplots as tsap

from polyphys.visualize import plotter
from polyphys.visualize import tuner
from polyphys.manage.parser import SumRule, TransFoci
from polyphys.manage import organizer

from polyphys.analyze import correlations 
from scipy import optimize

import inspect

# Finding Equilibirum porperties of Mon species: whole-level

In [ ]:
%%time
# Wall time: 1h 2min 23s for ns400nl5al5D20ac1
# loading databases:
database = '/Users/amirhsi_mini/research_data/analysis/'
#project_name = 'SumRule'
#spaces_to_read = ["N2000D30.0ac4.0","N2000D30.0ac6.0"]
#parser=SumRule
#properties = ['asphericityTMon', 'fsdTMon', 'gyrTMon', 'rfloryTMon', 'shapeTMon']

project_name = 'TransFoci'
spaces_to_read = ["ns400nl5al5D20ac1"]
#parser=TransFoci
properties = ['asphericityTMon', 'fsdTMon', 'gyrTMon', 'shapeTMon']

phase= "ens"
group = "bug"
species = "Mon"

chainsize = []



for space in spaces_to_read:
    space_db = database + "-".join([space, group, phase]) + "/*.csv" # path to ensembles
    ensembles = glob(space_db)
    # unique physical properties and their associated measures
    properties = set(
        [ens.split("/")[-1].split(group+"-")[-1].split(".csv")[0] \
         for ens in ensembles]
    )
    # uniques acfs of properties
    properties = [prop.split("-")[0] for prop in properties \
                  if prop.endswith("acf")]
    if project_name == 'TransFoci':
        properties.remove("rfloryTMon")
    properties = list(properties)
    properties.sort()
    for property_ in properties:
        print(property_)
        ensembles = glob(space_db)
        ensembles = organizer.sort_filenames(ensembles, fmts=[property_ + '.csv'])
        fit_df = correlations.fit_exp_wholes(
            space,
            space_db,
            property_,
            correlations.mono_exp_res,
            TransFoci,
            'biaxial',
            group,
            1,
            2,
            10,
            2000,
            x_property = 'index',
            save_to= "./",
            maxfev=20000
        )

### allInOne fits per space

In [ ]:
# loading databases:
database = '../Datasets'
#project_name = 'SumRule'
#spaces_to_read = ["N2000D30.0ac4.0","N2000D30.0ac6.0"]
#parser=SumRule
#parser=TransFoci
project_name = 'TransFoci'
spaces_to_read = ["ns400nl5al5D20ac1"]
group = "bug"
species = "Mon"

fit_db = "/".join([database, project_name, "fittings"])
fits = glob(fit_db + "/fitReport*")
fits_df = []
for fit_db in fits:
    fit_df = pd.read_csv(fit_db)
    fit_property = fit_db.split("-")[-1].split(".")[0]
    fit_df.rename(
        columns={'convergence': fit_property+'-convergence'}, inplace=True
    )
    fits_df.append(fit_df)

#### Merging space fit-report of different properties

In [ ]:
fits_df = pd.concat(fits_df, join='inner', axis=1)
fits_df = fits_df.loc[:,~fits_df.columns.duplicated()]
new_cols = {col: "".join(col.split("TMon")) if "TMon" in col else col for col in fits_df.columns}
fits_df.rename(
        columns=new_cols, inplace=True
    )
fits_df.replace([np.inf, -np.inf], np.nan, inplace=True)

#### Adding **phi_c_round** to the data set

In [ ]:
round_to = 0.025
phi_crds = fits_df.loc[:,'phi_c_bulk'].drop_duplicates().sort_values().values
phi_crds = np.round(np.round(phi_crds/round_to) * round_to, 3)
phi_crds = np.unique(phi_crds)
phi_crds.sort()
print(phi_crds)
print("Number of unique phi_c_bulk:", len(phi_crds))
rounding_func = lambda x, round_to: np.round(np.rint((x / round_to)) * round_to, 3)
fits_df['phi_c_bulk_round'] = fits_df['phi_c_bulk'].apply(
    rounding_func, args=[round_to]
)

In [ ]:
space_attributes = ['space', 'ensemble_long', 'ensemble',
       'nmon', 'dcyl', 'dcrowd',
       'phi_c_bulk', 'phi_c_bulk_round']

#### Normalizing different properties based on the physical limitation on them:

In [ ]:
cols_normlized = [col for col in fits_df.columns if col.endswith("mean") or col.endswith("residue")]
cols_max_by_phic_Zero = {}
round_to = 10
# finding the maximum of each property for phi_c_=0; the value of a property
# when phi_c=0 is the largest possible value of that property.
for property_ in cols_normlized:
    print(property_)
    norm_cond = fits_df['phi_c_bulk_round']==0
    normalizing_value = fits_df.loc[norm_cond, property_].max()
    cols_max_by_phic_Zero[property_] = np.ceil(normalizing_value/round_to) * round_to
    print(np.ceil(normalizing_value/round_to) * round_to)

In [ ]:
# The lower limit of different properties
cols_residues = [col for col in fits_df.columns if col.endswith("residue")]
for property_ in cols_residues:
    if property_ == 'shape-residue':
        fits_df.loc[fits_df[property_] < -1/4, property_] = 0
    else:
        fits_df.loc[fits_df[property_] < 0, property_] = 0

fits_df.loc[fits_df['asphericity-residue'] > 1, "asphericity-residue"] = 0
fits_df.loc[fits_df['shape-residue'] > 2, "shape-residue"] = 0

In [ ]:
# The uppoer limit of different properties
for property_ in cols_residues:
    if property_ != 'shape-residue' or property_ != 'asphericity-residue':
        fits_df.loc[fits_df[property_] > cols_max_by_phic_Zero[property_], property_] = cols_max_by_phic_Zero[property_]

#### Visualized comparison between fitting approach and ordinary averaging:

In [ ]:
sns.lineplot(
    x='phi_c_bulk_round',
    y='gyr-mean',
    data=fits_df
)

In [ ]:
sns.lineplot(
    x='phi_c_bulk_round',
    y='fsd-residue',
    data=fits_df  
)

# Finding Equilibirum porperties of Mon species: Space-level (or ensAvg level)

In [ ]:
space = spaces[1]
hue_attr = 'phi_c_bulk_round'
properties_mon = [ property_ for property_ in properties_specs if "Mon" in property_]
properties_mon_equil = ["".join(property_.split("T")) for property_ in properties_mon]
fitting_cols = properties_mon + ["time", "space", hue_attr]
fit_func = correlations.mono_exp_res
fit_params = inspect.getfullargspec(fit_func).args[1:]
hue_unique_values = phi_crds.copy()
s_info = parser(
        space, 
        geometry='biaxial',
        group=group,
        lineage='space',
        ispath=False
    )
space_title =  fr" $N={s_info.nmon}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
#space_title =  f" $n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, a_l={s_info.dmon_large}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
chainsize_space = chainsize.loc[chainsize['space']==space]

In [ ]:
#space_attributes = ['space', 'ensemble_long', 'ensemble',
#       'nmon_small', 'nmon_large', 'dmon_large', 'dcyl', 'dcrowd',
#       'phi_c_bulk', 'phi_c_bulk_round']

space_attributes = ['space', 'ensemble_long', 'ensemble',
       'nmon', 'dcyl', 'dcrowd',
       'phi_c_bulk', 'phi_c_bulk_round']

#### Fitting to ensembles in a space

##### report on the foolwing fit:
1. p0=[3,3,np.max(y),residue_guess] # This works for SumRule: N2000D30.0ac4.0 but the residue value of phic=0.350 is negative
2. p0=[2,2,np.max(y),np.min(y)] # This works for TransFoci: ns400nl5al5.0D20.0ac1.0
3. p0=[3,3,np.max(y),residue_guess] # This works for SumRule: N2000D30.0ac6.0 but the residue value of phic=0.350 is negative
    1. phic=0.25: could not fit 

In [ ]:
chainsize_quilibrium = chainsize_space[space_attributes].drop_duplicates()
chainsize_quilibrium.reset_index(inplace=True, drop=True)
chainsize_quilibrium["fitting_func"] = fit_func.__name__

chainsize_mon_properties = chainsize_space.loc[:,fitting_cols]
for property_ in properties_mon:
    chainsize_mon_properties[property_ + '-fit'] = 0
    for param in fit_params:
        param_name = "".join(property_.split("T"))+ "-" + param
        chainsize_quilibrium[param_name] = 0
        chainsize_quilibrium[param_name + "-StdError"] = 0
    print(property_)
    for hue_unique in hue_unique_values:
        print(hue_unique)
        attr_cond = chainsize_mon_properties[hue_attr] == hue_unique
        attr_cond_equil = chainsize_quilibrium[hue_attr] == hue_unique
        x = chainsize_mon_properties.loc[attr_cond, 'time'].to_numpy()
        y = chainsize_mon_properties.loc[attr_cond, property_].to_numpy()
        residue_guess = np.mean(y[-2000:])
        try:
            param_values, cov_mat = optimize.curve_fit(
            fit_func,
            x,
            y,
            p0=[3,3,np.max(y),residue_guess], # This works for SumRule: N2000D30.0ac4.0
            #p0=[2,2,np.max(y),np.min(y)] # This works for TransFoci: ns400nl5al5.0D20.0ac1.0
            maxfev=25000
            #bounds=((0,0.5,np.min(y),np.min(y)),(2,2,np.max(y),np.max(y)))
            )
            params_stds = np.sqrt(np.diag(cov_mat))
       
        except RuntimeError:
            print("could not fit ")
            param_values = np.zeros(4)
            params_stds = np.zeros(4)
            continue
        y_fit =  fit_func(x, *param_values)
        chainsize_mon_properties.loc[attr_cond, property_ + '-fit'] = y_fit
        # This part is fitting-function specific:
        for param, value, std_error in zip(fit_params, param_values, params_stds):
            param_name = "".join(property_.split("T"))+ "-" + param
            chainsize_quilibrium.loc[attr_cond_equil, param_name] = value
            chainsize_quilibrium.loc[attr_cond_equil, param_name + "-StdError"] = std_error

In [ ]:
save_to = "./"
ext = "csv"
output = "-".join(["tseries", "fittingData", project_name, space])
chainsize_mon_properties.to_parquet(save_to + output + ".parquet.brotli", compression="brotli", index=False)

#### Plotting data and fitting model

In [ ]:
legend_labels = {
    'gyrTMon': [r"$R_g(\hat{t})$", r"$R_g^{(fit)}(\hat{t})$"],
    'fsdTMon': [r"$L(\hat{t})$", r"$L^{(fit)}(\hat{t})$"],
    'asphericityTMon': [r"$\Delta(\hat{t})$", r"$\Delta^{(fit)}(\hat{t})$"],
    'shapeTMon': [r"$S(\hat{t})$", r"$S^{(fit)}(\hat{t})$"],
    'rfloryTMon': [r"$R_F(\hat{t})$", r"$R_F^{(fit)}(\hat{t})$"]
}
properties_mon_equil_labels = {
    'gyrMon': {
        "symbol": r"$\bar{R}_g$",
        "norm": r"$\frac{\bar{R}_g}{\bar{R}_{g,0}}$",
        "title": r"$\bar{R}_g(\phi_c^{(bulk)})$"
    },
    'fsdMon': {
        "symbol": r"$\bar{L}$",
        "norm": r"$\frac{\bar{L}}{\bar{L}_0}$",
        "title": r"$\bar{L}(\phi_c^{(bulk)})$"
    },
    'asphericityMon': {
        "symbol": r"$\bar{\Delta}$",
        "norm": r"$\frac{\bar{\Delta}}{\bar{\Delta}_0}$",
        "title": r"$\bar{\Delta}(\phi_c^{(bulk)})$"
    },
    'shapeMon': {
        "symbol": r"$\bar{S}$",
        "norm": r"$\frac{\bar{S}}{\bar{S}_0}$",
        "title": r"$\bar{S}(\phi_c^{(bulk)})$"
    },
    'rfloryMon': {
        "symbol": r"$\bar{R}_F$",
        "norm": r"$\frac{\bar{R}_F}{\bar{R}_{F,0}}$",
        "title": r"$\bar{R}_F(\phi_c^{(bulk)})$"
    }
}

In [ ]:
col_attr = 'phi_c_bulk_round'
fontsize=26
ext = 'pdf'
for property_ in properties_mon:
    property_equil_name = "".join(property_.split("T"))
    chainsize_space_melted = chainsize_mon_properties[['time', 'phi_c_bulk_round',property_, property_+"-fit"]].melt(id_vars=['time','phi_c_bulk_round'],value_vars=[property_, property_+"-fit"])
    sns.set_context(
            font_scale=3,
            rc={
                'font.family': "Times New Roman",
                'mathtext.default': 'regular',
                "text.usetex": True,
                "font.size": fontsize
            }
        )
    tseries_fit_fgrid = sns.FacetGrid(
        chainsize_space_melted,
        col="phi_c_bulk_round",
        col_wrap=3,
        height=6,
        aspect=16/9,
        legend_out=False
    )
    tseries_fit_fgrid.map_dataframe(
        sns.lineplot,
        x="time",
        y="value",
        hue='variable',
        size='variable',
        size_order=[property_+"-fit", property_],
        ci=None,
        estimator=None)
    new_labels = legend_labels[property_]
    for ax, hue_unique in zip(tseries_fit_fgrid.axes.flat,hue_unique_values):
        long_time_value = np.round(
            chainsize_quilibrium.loc[chainsize_quilibrium[hue_attr]==hue_unique,property_equil_name+"-residue"].values[0],
            2
        )
        #label_value = mean_labels[property_].format(long_time_value)
        ax.axhline(
            long_time_value,
            xmin=0.02,
            xmax=0.98,
            color="black",
            lw=2,
            alpha=0.7,
            label= properties_mon_equil_labels[property_equil_name]['symbol'],
            linestyle="-.")

        handles, labels = ax.get_legend_handles_labels()
        new_labels.append(labels[-1])
        ax.legend(handles = handles, labels=new_labels, frameon=False, ncol=3)
        new_labels = legend_labels[property_]

    tseries_fit_fgrid.set_axis_labels(
        titles['time'], properties_specs[property_]['symbol'])
    tseries_fit_fgrid.set_titles(
        titles[col_attr]+r"$={col_name}$")
    tseries_fit_fgrid.tight_layout(w_pad=1)
    output = "-".join(["tseries-fit", project_name, property_, space]) + "." + ext
    tseries_fit_fgrid.savefig(save_to + output, bbox_inches='tight')
    plt.close()

#### Extimate equilibrium properties from fitting model

In [ ]:
for property_ in properties_mon_equil:
    chainsize_quilibrium[property_ + "-norm"] = chainsize_quilibrium[property_+ "-residue"] / chainsize_quilibrium.loc[chainsize_quilibrium[hue_attr]==0, property_+ "-residue"].values[0]

In [ ]:
output = "-".join(["tseries", "equilibriumValues", project_name, space])
chainsize_quilibrium.to_csv(save_to + output + ".csv", index=False)

In [ ]:
chainsize_quilibrium

#### Equilibrium plots all in one

In [ ]:
properties_mon_equil_norm = [property_ + "-norm" for property_ in properties_mon_equil]
selected_cols = space_attributes + properties_mon_equil_norm
chainsize_quilibrium_melted = chainsize_quilibrium[selected_cols].melt(id_vars=space_attributes,value_vars=properties_mon_equil_norm)

In [ ]:
fontsize=16
s_info = parser(
        space, 
        geometry='biaxial',
        group=group,
        lineage='space',
        ispath=False
    )
space_title =  fr" $N={s_info.nmon}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
#space_title =  f" $n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, a_l={s_info.dmon_large}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
sns.set_context(
            font_scale=2,
            rc={
                'font.family': "Times New Roman",
                'mathtext.default': 'regular',
                "text.usetex": True,
                "font.size": fontsize
            }
        )
tseries_fit_fgrid = sns.relplot(
    x=hue_attr,
    y='value',
    col='variable',
    color='steelblue',
    kind='line',
    marker="s",
    markersize=7,
    ls=":",
    data=chainsize_quilibrium_melted,
    col_wrap=2,
    facet_kws = {'sharey': False, 'sharex': False}
)
for ax, property_ in zip(tseries_fit_fgrid.axes.flat,properties_mon_equil):
        ax.set_ylabel(
            properties_mon_equil_labels[property_]["norm"],
            rotation=0,
            labelpad=20
        )
        ax.set_title(None)
tseries_fit_fgrid.set_xlabels(titles[col_attr])
tseries_fit_fgrid.tight_layout(w_pad=0)
tseries_fit_fgrid.fig.suptitle(space_title, y=1.05)
output = "-".join(["equilPlot-allInOne", project_name, space]) + "." + ext
tseries_fit_fgrid.savefig(save_to + output, bbox_inches='tight')
plt.close()

#### Single equilibrium plots 

In [ ]:
fontsize=16
ext='pdf'
save_to = './'
property_ = 'gyrMon'
s_info = parser(
        space, 
        geometry='biaxial',
        group=group,
        lineage='space',
        ispath=False
    )
space_title =  fr" $N={s_info.nmon}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
#space_title =  f" $n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, a_l={s_info.dmon_large}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
for property_ in properties_mon_equil:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,4.5))
    sns.set_context(
            font_scale=2,
            rc={
                'font.family': "Times New Roman",
                'mathtext.default': 'regular',
                "text.usetex": True,
                "font.size": fontsize
            }
        )
    line_ax = sns.lineplot(
        x=hue_attr,
        y=property_ + "-norm",
        marker="o",
        ls=":",
        data=chainsize_quilibrium,
        ax=ax,
    )
    ax.set_ylabel(
        properties_mon_equil_labels[property_]['norm'],
        rotation=0,
        labelpad=20
    )
    ax.set_xlabel(titles[hue_attr])
    ax.set_title(space_title)
    output = "-".join(["equilPlot", project_name, space,property_,hue_attr]) + "." + ext
    fig.savefig(save_to + output, bbox_inches='tight')
    plt.close()